## Initialization

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats import chi2, kstest, norm
import seaborn as sns
import math
import scipy.stats as stats

## 1) Crude Monte Carlo

In [11]:
np.random.seed(1234)

def MonteCarlo(n):
    U = np.random.uniform(0, 1, n)
    X = np.exp(U)
    mean = np.mean(X)
    std = np.std(X)
    standard_error = std / np.sqrt(n)
    confidence_interval = (mean - 1.96 * standard_error, mean + 1.96 * standard_error)
    return mean, confidence_interval


mean, confidence_interval = MonteCarlo(100)
print(mean)
print(np.round(confidence_interval,2))



1.743309935963331
[1.65 1.84]


## Antithetic Variables

In [13]:
def AntitheticVariables(n):

    U = np.random.uniform(0, 1, n//2)

    combined_samples = np.concatenate([U, 1-U])

    X = np.exp(combined_samples)

    mean = np.mean(X)
    std = np.std(X)
    standard_error = std / np.sqrt(n)
    confidence_interval = (mean - 1.96 * standard_error, mean + 1.96 * standard_error)

    return mean, confidence_interval


mean, confidence_interval = AntitheticVariables(100)
print(mean)
print(np.round(confidence_interval,2))

1.7282853392863842
[1.63 1.83]


## Control Varible

In [14]:
def ControlVariables(n):

    U = np.random.uniform(0, 1, n//2)

    X = np.exp(U)
    control_var = U

    expected_control_var = 0.5

    alpha = -1  
    control_variate_estimator = X + alpha * (control_var - expected_control_var)

    mean = np.mean(control_variate_estimator)

    std = np.std(control_variate_estimator)

    standard_error = std / np.sqrt(n)

    confidence_interval = (mean - 1.96 * standard_error, mean + 1.96 * standard_error)

    return mean, confidence_interval


mean, confidence_interval = ControlVariables(100)
print(mean)
print(np.round(confidence_interval,2))

1.7765263059216747
[1.74 1.82]


## Stratified Sampling

In [15]:
def StratifiedSampling(n):

    k = 10  # number of strata
    samples_per_stratum = n // k

    # Generate stratified samples
    stratified_samples = []
    for i in range(k):
        stratum_samples = np.random.uniform(i / k, (i + 1) / k, samples_per_stratum)
        stratified_samples.extend(stratum_samples)

    U = np.array(stratified_samples)

    X = np.exp(U)

    mean = np.mean(X)

    std = np.std(X)

    standard_error = std / np.sqrt(n)

    confidence_interval = (mean - 1.96 * standard_error, mean + 1.96 * standard_error)

    return mean, confidence_interval


mean, confidence_interval = ControlVariables(100)
print(mean)
print(np.round(confidence_interval,2))

1.7097066831022851
[1.67 1.75]


## Summary of methods

In [28]:
n = 100

print('True Mean Value:')
print('1.72 \n')

print('Monte Carlo:')
mean, confidence_interval = MonteCarlo(n)
print(np.round(mean,4),'          ',np.round(confidence_interval,2),'        ',np.round(np.abs(mean-1.72),4),'\n')

print('Antithetic Variables:')
mean, confidence_interval = AntitheticVariables(n)
print(np.round(mean,4),'          ',np.round(confidence_interval,2),'        ',np.round(np.abs(mean-1.72),4),'\n')

print('Control Variables:')
mean, confidence_interval = ControlVariables(n)
print(np.round(mean,4),'          ',np.round(confidence_interval,2),'        ',np.round(np.abs(mean-1.72),4),'\n')

print('Stratified Sampling:')
mean, confidence_interval = StratifiedSampling(n)
print(np.round(mean,4),'          ',np.round(confidence_interval,2),'        ',np.round(np.abs(mean-1.72),4),'\n')

True Mean Value:
1.72 

Monte Carlo:
1.7352            [1.64 1.84]          0.0152 

Antithetic Variables:
1.7192            [1.62 1.82]          0.0008 

Control Variables:
1.7311            [1.7  1.77]          0.0111 

Stratified Sampling:
1.7187            [1.62 1.82]          0.0013 



## Redusing Variance of Exercise 4 with control variates

In [58]:
# Parameters
m = 10                  
mst = 8   
mtbc = 1 
N = 100000  
n= 10   

# set seed
np.random.seed(123)

def Exp_service_time(mst):
    return np.random.exponential(mst)

# Function to run a single simulation 
def Run_Poisson_Sim(m,mst,mtbc,n,N,service_dist):
    """
    Runs a simulation of a blocking system with m service units.
    
    Input: 
    m = Service units 
    mst = Mean Service time 
    mtbc = mean time between customers
    n = Number of simulations 
    N = number of customers

    Output: 
    - Returns the fraction of blocked customers 
    """
    # Known expected value of the control variate (total idle time)
    expected_service_time = N*mst

    blocked_fractions = []
    total_service_time = []
    
    for _ in range(n):
        # Initialize parameters 
        t = 0
        blocked_customers = 0
        tot_ser_time = 0
        busy_until = [0] * m

        # loop over number of customers 
        for _ in range(N):
            # Generate next arrival time
            t += np.random.exponential(mtbc)
            
            # Check for an available server
            available_server = -1
            for i in range(m):
                if busy_until[i] <= t:
                    available_server = i
                    break
            
            if available_server == -1:
                # All servers are busy
                blocked_customers += 1
            else:
                # Assign the customer to the available server
                service_time = service_dist()
                tot_ser_time += service_time
                busy_until[available_server] = t + service_time


        blocked_fractions.append(blocked_customers / N)
        total_service_time.append(tot_ser_time)
    

    mean_blocked = np.mean(blocked_fractions)
    mean_service_time = np.mean(total_service_time)

    

    variance_control_var = np.var(total_service_time)
    if variance_control_var == 0:
        raise ValueError("Control variable has zero variance, choose a different control variate.")

    cov = np.cov(blocked_fractions, total_service_time)[0, 1]
    var = np.var(total_service_time)
    alpha = cov / var

    adjusted_blocking_fractions = [
    bf - alpha * (cv - expected_service_time)
    for bf, cv in zip(blocked_fractions, total_service_time)
    ]  

    #Variance: 
    varX = np.var(blocked_fractions)

    varZ = varX - ((cov**2)/var )

    print('variance of X: ', varX)
    print('variance of Z: ', varZ)

    mean = np.mean(adjusted_blocking_fractions)
    std= np.std(adjusted_blocking_fractions)
    conf_int = stats.norm.interval(0.95, loc=mean, scale=std / np.sqrt(n))

    return adjusted_blocking_fractions,mean, conf_int

def Erlang_B(A,m):
    # Analytical calculation using Erlang's B-formula
    return A**m / math.factorial(m)/ sum(A**i / math.factorial(i) for i in range(m + 1))

generator = lambda: Exp_service_time(mst)
# Run simulations
blocked_fractions_1,mean_1, conf_int_1 = Run_Poisson_Sim(m,mst, mtbc,n,N,generator) 



variance of X:  2.421484000000001e-06
variance of Z:  2.147183138949091e-06


In [66]:
#Function for hyper exponential distribution
def hyperexponential(p1, lambda1, p2, lambda2):
    
    if np.random.rand() < p1:
        np.random.seed(42)
        return np.random.exponential(1/lambda1)
    else:
        np.random.seed(42)
        return np.random.exponential(1/lambda2)

# Function to run a single simulation 
def CommonRandomNumbers(m,mst,mtbc,n,N,service_dist):
    
    blocked_fractions_poi = []
    blocked_fractions_hype = []
    
    # Hyper-exponential distributed inter-arrival times
    p1, lambda1 = 0.8, 0.8333
    p2, lambda2 = 0.2, 5.0

    for _ in range(n):
        # Initialize parameters 
        t_poi = 0
        blocked_customers_poi = 0
        busy_until_poi = [0] * m

        t_hyp = 0
        blocked_customers_hyp = 0
        busy_until_hyp = [0] * m

        # loop over number of customers 
        for _ in range(N):
            # Generate next arrival time
            np.random.seed(42)
            t_poi += np.random.poisson(mtbc)
            t_hyp += hyperexponential(p1, lambda1, p2, lambda2)
            
            # Check for an available server
            available_server_poi = -1
            for i in range(m):
                if busy_until_poi[i] <= t_poi:
                    available_server_poi = i
                    break
            
            if available_server_poi == -1:
                # All servers are busy
                blocked_customers_poi += 1
            else:
                # Assign the customer to the available server
                service_time = service_dist()
                busy_until_poi[available_server_poi] = t_poi + service_time

            #_____________________# 

            # Check for an available server
            available_server_hyp = -1
            for i in range(m):
                if busy_until_hyp[i] <= t_hyp:
                    available_server_hyp = i
                    break
            
            if available_server_hyp == -1:
                # All servers are busy
                blocked_customers_hyp += 1
            else:
                # Assign the customer to the available server
                service_time = service_dist()
                busy_until_hyp[available_server_hyp] = t_hyp + service_time
        
        blocked_fractions_poi.append(blocked_customers_hyp / N)
        blocked_fractions_hype.append(blocked_customers_hyp / N)
    
    
    mean_poi = np.mean(blocked_fractions_poi)
    mean_hype = np.mean(blocked_fractions_hype)
    diff = mean_poi - mean_hype
    std= np.std(np.array(blocked_fractions_poi)-np.array(blocked_fractions_hype))
    conf_int = stats.norm.interval(0.95, loc=diff, scale=std / np.sqrt(n))

    return np.array(blocked_fractions_poi)-np.array(blocked_fractions_hype), diff, conf_int

generator = lambda: Exp_service_time(mst)
# Run simulations
blocked_fractions_1,mean, conf_int = CommonRandomNumbers(m,mst, mtbc,n,N,generator) 

# Print results
print(f"Mean: {mean:.4f}")
print(f"95% confidence interval: ({conf_int[0]:.4f}, {conf_int[1]:.4f})")

# No difference ??? 

Mean: 0.0000
95% confidence interval: (nan, nan)


## Exercise 7

In [67]:
import numpy as np

# Crude Monte Carlo estimator for P(Z > a)
def crude_monte_carlo(a, num_samples):
    samples = np.random.randn(num_samples)  # generate standard normal samples
    probability = np.mean(samples > a)
    return probability

# Importance sampling
def importance_sampling(a, sigma, num_samples):
    samples = np.random.normal(loc=a, scale=sigma, size=num_samples)  # generate samples from normal density with mean a and variance sigma^2
    weights = np.exp(-0.5 * ((samples - a) / sigma) ** 2) / (sigma * np.sqrt(2 * np.pi))  # compute importance weights
    probability = np.mean((samples > a) * weights) / np.mean(weights)  # estimate probability using importance weights
    return probability

# Experimentation
sample_sizes = [1000, 5000, 10000]  # different sample sizes
a_values = [2, 4]  # different values of 'a'
sigma_values = [1]  # different values of sigma^2

for sigma in sigma_values:
    print(f"Sigma^2 = {sigma}")
    for a in a_values:
        print(f"\nFor a = {a}:")
        for num_samples in sample_sizes:
            print(f"\nSample size = {num_samples}:")
            # Crude Monte Carlo
            crude_prob = crude_monte_carlo(a, num_samples)
            print("Crude Monte Carlo Estimator:", crude_prob)
            # Importance Sampling
            importance_prob = importance_sampling(a, sigma, num_samples)
            print("Importance Sampling Estimator:", importance_prob)

# Efficiency Discussion
print("\nEfficiency Discussion:")
print("Importance sampling tends to be more efficient when the tails of the distribution are important. It achieves this by assigning higher probabilities to rare events, thereby reducing variance compared to crude Monte Carlo.")

Sigma^2 = 1

For a = 2:

Sample size = 1000:
Crude Monte Carlo Estimator: 0.024
Importance Sampling Estimator: 0.5096938845613809

Sample size = 5000:
Crude Monte Carlo Estimator: 0.022
Importance Sampling Estimator: 0.49543485671304904

Sample size = 10000:
Crude Monte Carlo Estimator: 0.0229
Importance Sampling Estimator: 0.4996020353681839

For a = 4:

Sample size = 1000:
Crude Monte Carlo Estimator: 0.0
Importance Sampling Estimator: 0.4951835287861722

Sample size = 5000:
Crude Monte Carlo Estimator: 0.0
Importance Sampling Estimator: 0.5080669779410953

Sample size = 10000:
Crude Monte Carlo Estimator: 0.0
Importance Sampling Estimator: 0.5045255348948406

Efficiency Discussion:
Importance sampling tends to be more efficient when the tails of the distribution are important. It achieves this by assigning higher probabilities to rare events, thereby reducing variance compared to crude Monte Carlo.


## Exercise 8

In [104]:
U = np.random.uniform(0, 1, num_samples)
X = np.exp(U)
mean = np.mean(X)
print(mean)
# Original function f(x) = e^x
def f(x):
    return np.exp(x)

# Importance sampling function g(x) = lambda * exp(-lambda * x)
def g(x, lmbda):
    return lmbda * np.exp(-lmbda * x)

# Importance sampling estimator
def importance_sampling_estimator(lmbda, num_samples):
    samples = np.random.exponential(scale=1/lmbda, size=num_samples)  # generate samples from exponential distribution
    weights = f(samples) / g(samples, lmbda)  # compute importance weights
    estimate = np.mean(weights)
    return estimate

# Parameters
lmbda = 1  # lambda value for the exponential distribution
num_samples = 100000  # number of samples

# Estimate integral using importance sampling
estimate = importance_sampling_estimator(lmbda, num_samples)
print("Estimated value:", estimate)

1.7177326610281662
Estimated value: 34096305.04034875
